# REST API RAG Samples

## Objective
Enhancing capabilities by bringing custom data to augment image inputs in GPT-4V.

## Time

You should expect to spend 5-10 minutes running this sample.

## Before you begin

#### Installation

In [ ]:
%pip install -r ../requirements.txt

### Parameters
You need to set a series of configurations such as GPT-4V_DEPLOYMENT_NAME, OPENAI_API_BASE, OPENAI_API_VERSION, VISION_API_ENDPOINT, AZURE_SEARCH_SERVICE_ENDPOINT, AZURE_SEARCH_INDEX_NAME.
```
Add "OPENAI_API_KEY", "VISION_API_KEY", and "AZURE_SEARCH_QUERY_KEY" as variable name and \<Your API Key Value\>, \<Your VISION Key Value\>, and \<Your SEARCH Query Key Value\> as variable value in the environment variables.
 <br>
      
      WINDOWS Users: 
         setx OPENAI_API_KEY "REPLACE_WITH_YOUR_KEY_VALUE_HERE"
         setx VISION_API_KEY "REPLACE_WITH_YOUR_KEY_VALUE_HERE"
         setx AZURE_SEARCH_QUERY_KEY "REPLACE_WITH_YOUR_KEY_VALUE_HERE"

      MACOS/LINUX Users: 
         export OPENAI_API_KEY="REPLACE_WITH_YOUR_KEY_VALUE_HERE"
         export VISION_API_KEY="REPLACE_WITH_YOUR_KEY_VALUE_HERE"
         export AZURE_SEARCH_QUERY_KEY="REPLACE_WITH_YOUR_KEY_VALUE_HERE"

In [ ]:
# Setting up the deployment name
deployment_name: str = "<your GPT-4V deployment name>"
# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base: str = "<your resource base URL>"
# Currently OPENAI API have the following versions available: 2022-12-01.
# All versions follow the YYYY-MM-DD date structure.
openai_api_version: str = "<your OpenAI API version>"

# The base URL for your vision resource endpoint, e.g. "https://<your-resource-name>.cognitiveservices.azure.com"
vision_api_endpoint: str = "<your vision resource endpoint>"

# Setting up the Azure Search service endpoint.e.g. https://<your search service name>.search.windows.net
search_service_endpoint: str = "<your search service endpoint>"
# Setting up the Azure Search service index
search_index_name: str = "<your search index name>"

should_cleanup: bool = False

## Connect to your project
To start with let us create a config file with your project details. This file can be used in this sample or other samples to connect to your workspace.

In [ ]:
import json
from pathlib import Path

config = {
    "GPT-4V_DEPLOYMENT_NAME": deployment_name,
    "OPENAI_API_BASE": openai_api_base,
    "OPENAI_API_VERSION": openai_api_version,
    "VISION_API_ENDPOINT": vision_api_endpoint,
    "AZURE_SEARCH_SERVICE_ENDPOINT": search_service_endpoint,
    "AZURE_SEARCH_INDEX_NAME": search_index_name,
}

p = Path("../config.json")

with p.open(mode="w") as file:
    file.write(json.dumps(config))

## Run this Example

### Create Azure Search Index

In [ ]:
# Using the Azure Search service create the index with image embeddings
# https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/azure-search-vector-image-index-creation-python-sample.ipynb

### Call GPT-4V API with Image

In [ ]:
import os
import base64
from IPython.display import Image, display
import sys

parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from shared_functions import call_GPT4V_image

# Setting up the vision resource key
vision_api_key = os.getenv("VISION_API_KEY")

# Setting up the Azure Search service query key
search_query_key = os.getenv("AZURE_SEARCH_QUERY_KEY")

# System messages and user prompt
sys_message = "You are an AI assistant that helps people find information."
user_prompt = "What are the types of the apple(s) shown in this image?"

# Encode the image in base64
image_file_path = "test_Gala.jpg"
with Path(image_file_path).open("rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

messages = [
    {"role": "system", "content": [{"type": "text", "text": sys_message}]},
    {
        "role": "user",
        "content": [
            {"type": "text", "text": user_prompt},  # Prompt for the user
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"},  # Image to be processed
            },
        ],
    },
]

in_context_config = {"endpoint": search_service_endpoint, "key": search_query_key, "indexName": search_index_name}

vision_api_config = {"endpoint": vision_api_endpoint, "key": vision_api_key}

try:
    response_content = call_GPT4V_image(messages, in_context=in_context_config, vision_api=vision_api_config)
    display(Image(image_file_path))
    print(response_content["choices"][0]["message"]["content"])  # Print the content of the response
except Exception as e:
    print(f"Failed to call GPT-4V API. Error: {e}")

## Cleaning up

To clean up all Azure ML resources used in this example, you can delete the individual resources you created in this tutorial.

If you made a resource group specifically to run this example, you could instead [delete the resource group](https://learn.microsoft.com/en-us/azure/azure-resource-manager/management/delete-resource-group).

In [ ]:
if should_cleanup:
    # {{TODO: Add resource cleanup}}
    pass